#  Sur les trames NMEA

<div class="alert alert-info">
    
### À propos de ce document (notebook Jupyter)

Un notebook Jupyter est un document utilisable dans un navigateur internet et qui offre un noyau Python pour faire un peu de programmation informatique.  
Vous verrez plusieurs types de **cellules** :
* Cellules comme celle-ci, textes à lire, énoncés uniquement  ;
* Cellules Python pour faire un peu d'informatique. Elle sont précédée en marge d'un In[ ] ou Entrée[ ]. Il faudra parfois y compléter certaines parties, indiquée par un commentaire # à compléter. Il faudra les exécuter en appuyant simultanément sur les touches "ctrl" et "entrée".
* Cellules de réponses. Il faut y écrire les réponses.
</div>

<div class="alert alert-info">

### Document 1 : les trames NMEA - présentation

Il existe plusieurs protocoles de communication pour les récepteurs GPS. Le plus courant est le protocole **NMEA 0183**.  
Ce protocole échange les informations dans des **trames** normalisées. Elles sont générées par le récepteur GPS à partir des informations reçues depuis les satellites et partagées ensuite aux applications utilisant la localisation.  
Chaque trame commence par le caractère '&#36;' et finit par un passage à la ligne (RC-LF).  
L'intérieur de la trame est constitué de différents champs séparés par des virgules. Le dernier champ est constitué d'une * suivant d'un _checksum_ qui permet de vérifier l'intégritée de la transmission.  
Le premier champ après &#36; est l'identifiant : il est constitué de 2 lettres indiquant la source du signal
- BD ou GB pour Beidou
- GA pour Galileo
- GP pour GPS
- GL pour GLONASS
- ...

suivi d'un code de 3 lettres indiquant le type de trame. Il en existe plus de 70 différents, selon l'utilisation qu'on en fait. Pour déterminer une position au sol, les plus intéressants sont
* GGA
* RMC
* GLL  

Plus d'information sur : https://fr.wikipedia.org/wiki/NMEA_0183
</div>

### Exercice 1

On a recueilli les trois trames NMEA suivantes :
1. &#36;GPRMC,150726.355,A,4800.0035,N,00242.8533,E,19.1,106.1,281219,,,A*56
2. &#36;GPGGA,150733.000,4851.4909,N,00217.6657,E,1,15,0.6,131.9,M,47.9,M,,*59
3. &#36;GPGLL,4754.11066,N,00154.62334,E,225444,A,*1D

**Question 1**  
Identifier la trame de type GGA.

**Question 2**  
En utilisant le document ci-dessous, identifier la latitude et la longitude du lieu correspondant à cette trame.

<div class="alert alert-info">

### Document 2 : constitution d'une trame GGA

trame : &#36;GPGGA,123519,4807.038,N,01131.000,E,1,08,0.9,545.4,M,46.9,M,,*47  
Where:
* GGA          Global Positioning System Fix Data
* 123519       Fix taken at 12:35:19 UTC
* 4807.038,N   Latitude 48 deg 07.038' N
* 01131.000,E  Longitude 11 deg 31.000' E
* 1            Fix quality
* 08           Number of satellites being tracked
* 0.9          Horizontal dilution of position
* 545.4,M      Altitude, Meters, above mean sea level
* 46.9,M       Height of geoid (mean sea level) above WGS84 ellipsoid
* (empty field) time in seconds since last DGPS update
* (empty field) DGPS station ID number
* *47          the checksum data, always begins with *
</div>

**Question 3**  
Dans l'exemple du document 2, la latitude est de $la = 48°07.038'$ N.  
Comme $1°=60'$, $la = 48+\left(07.038/60 \right) = 48.1173°$  
Calculer de la même manière la latitude et la longitude en ° décimaux du lieu correspondant à la trame 2.

**Question 4**  
Nous allons utiliser une module python `folium` qui permet de visualiser sur des cartes.  
Dans la cellule ci-dessous, compléter les coordonnées `lat` et `lon` par les valeurs que vous avez trouvé puis exécuter la cellule (ctrl + entrée).

In [ ]:
import folium         # module permettant de manipuler une carte numérique

lat =  # à compléter
lon =  # à compléter

point = (lat , lon)   # Définition du point cherché

carte = folium.Map(location=point,zoom_start=12)   # création de la carte centrée sur le point

folium.Marker(point, popup = 'lieu cherché').add_to(carte)    # positionnement d'un marqueur au point

carte               # Affichage de la carte

**Question 5**  
En zoomant sur la carte, identifier le lieu remarquable qui est ainsi identifié.

**Question 6**  
Reprendre les étapes précédentes pour identifier de la même manière les bâtiments correspondants aux deux autres lieux.

### Exercice 2

Nous allons faire un peu de programmation pour automatiser le traitement (ne pas calculer à la main la latitude et la longitude.  
La trame MNEA est une chaîne de caractères, qu'il faut découper etdont il faut extraire les informations qui nous intéressent.
```python
    "$GPGGA,101316.49,4041.3514,N,07402.6736,W,0,,100.000,120.064,M,,M,0,*4A"
    "$GPRMC,053740.000,A,2503.6319,N,12136.0099,E,2.69,79.65,100106,,,A*53"
```
Les différentes étapes :
* convertir les degrés, minutes en degrés décimaux
* selon le type, aller chercher la latitude et la longitude au bon endroit
* identifier le type

**Question 1**  
Nous allons commencer par la conversion.  
Compléter la fonction python suivante avec la formule de calcul utilisée précédemment, puis exécuter la cellule.

In [ ]:
def conversion_degreminute_degredecimaux(degre, minute):
    degre_decimaux =        # à compléter
    return degre_decimaux

# Pour tester la fonction : on doit obtenir environ 48.858182
conversion_degreminute_degredecimaux(48, 51.4909)

**Question 2**  
Pour traiter la trame et aller chercher la latitude et la longitude nous devont la casser et en extraire des morceaux.  
Pour la casser, nous utiliserons la fonction `split(séparateur)`.  
Pour en extraire des morceaux, nous utiliserons `trame[p:q]` qui va chercher les caractères entre la position p et la position (q-1).  
Pour cela, nous avons écrit les deux fonctions `coordonnees_GGA` et `coordonnees_RMC`.  
Exécuter la cellule suivante.

In [ ]:
def coordonnees_GGA(trame):
    latitude, lat_dir, longitude, lon_dir = trame.split(',')[2:6]
    latitude = conversion_degreminute_degredecimaux(int(latitude[0:2]), float(latitude[2:]))
    if lat_dir=="S":
        latitude = -latitude
    longitude = conversion_degreminute_degredecimaux(int(longitude[0:3]), float(longitude[3:]))
    if lon_dir=="W":
        longitude = -longitude
    return (latitude, longitude)


def coordonnees_RMC(trame):
    latitude, lat_dir, longitude, lon_dir = trame.split(',')[3:7]
    latitude = conversion_degreminute_degredecimaux(int(latitude[0:2]), float(latitude[2:]))
    if lat_dir=="S":
        latitude = -latitude
    longitude = conversion_degreminute_degredecimaux(int(longitude[0:3]), float(longitude[3:]))
    if lon_dir=="W":
        longitude = -longitude
    return (latitude, longitude)

Pour identifier le type, nous devons extraire dans la chaîne les caractères en positions 3, 4 et 5 (car le premier caractère &#36; est à la position 0.    
Ensuite, il faut comparer avec `"GGA"` ou `"RMC"` (ou dans l'absolu toutes les autres possibilités) pour savoir de quel type il s'agit et faire appel à la fonction correspondante dans les deux fonctions précédentes.  
Placer au bon endroit chacun des appels suivants :
```python
coordonnees_RMC(trame)
coordonnees_GGA(trame)
```
et exécuter la cellule.

In [ ]:
def type_trame(trame):
    ttype = trame[3:6]
    if ttype == "GGA":
        point =          # à compléter
    elif ttype == "RMC":
        point =          # à compléter
    else:
        point = None
    return point

**Question 3**  
Il nous reste à tester avec chacune de nos trames.  
Exécuter le cellule suivante et trouver le monument célèbre qui correspond à cette trame.

In [ ]:
mon_point = type_trame("$GPGGA,101316.49,4041.3514,N,07402.6736,W,0,,100.000,120.064,M,,M,0,*4A")

carte = folium.Map(mon_point, zoom_start = 10)
folium.Marker(mon_point, popup='Quel monument ?').add_to(carte)

carte

Monument : 

**Question 4**  
Seconde trame ; où cela nous mène-t-il ?

In [ ]:
mon_point = type_trame("$GPRMC,053740.000,A,2503.6319,N,12136.0099,E,2.69,79.65,100106,,,A*53")

carte = folium.Map(mon_point, zoom_start = 10)
folium.Marker(mon_point, popup='où est-ce ?').add_to(carte)

carte

### Exercice 3.

Nous allons tracer le cercle de 100 km autour de chez vous dans lequel vous avez le droit de circuler dans cette phase de déconfinement.  
Déterminer les coordonnées GPS de votre maison.  
Compléter la cellule ci-dessous avec les valeurs trouvées et exécutez la.

In [ ]:
latitude =    # à compléter
longitude =    # à compléter

maison = (latitude, longitude)
  
ma_carte = folium.Map(location=maison, zoom_start=7)

# ajouter le cercle des 100 km
folium.Circle(
        location = maison,
        radius = 100000,
        color = '#880000',
        fill = True
    ).add_to(ma_carte)

folium.Marker(maison, popup = 'maison').add_to(ma_carte)

ma_carte

## Rendu du travail.

Sauvegarder votre travail de la manière suivante :
1. Assurer vous d'avoir complété toutes les réponses.
2. Menu Noyau
>* Redémarrer et effacer les sorties
>* Relancer et effacer toutes les sorties dans la boîte de dialogue
3. Menu fichier
>* Télécharger au format
>* choisir Notebook
>* Envoyer le fichier ainsi téléchargé (trame-nmea.ipynb) par mail à herve.vasseur@ac-orleans-tours.fr
